In [28]:
from andi_datasets.datasets_phenom import datasets_phenom
datasets_phenom().avail_models_name

import os
import numpy as np

# Remains to be computed : time spent in each state; how do we differentiate multi rom dimer from confinement ?

In [29]:
def analyze_diffusion_coefficients(K, alpha, states, num_states, model):
    K = np.array(K)
    alpha = np.array(alpha)
    states = np.array(states)
    
    mean_alphas = []
    std_alphas = []
    mean_Ks = []
    std_Ks = []
    N = []
    
    for state in range(num_states):
        state_indices = np.where(states == state)[0]
        
        if len(state_indices) > 0:
            mean_alphas.append(np.mean(alpha[state_indices]))
            std_alphas.append(np.std(alpha[state_indices]))
            mean_Ks.append(np.mean(K[state_indices]))
            std_Ks.append(np.std(K[state_indices]))
            N.append(len(state_indices))
        else:
            mean_alphas.append(np.nan)
            std_alphas.append(np.nan)
            mean_Ks.append(np.nan)
            std_Ks.append(np.nan)
            N.append(0)
    
    # Prepare the result string
    result = f"model: {model}; num_state: {num_states} \n"
    result += "; ".join(map(str, mean_alphas)) + ";\n"
    result += "; ".join(map(str, std_alphas)) + ";\n"
    result += "; ".join(map(str, mean_Ks)) + ";\n"
    result += "; ".join(map(str, std_Ks)) + ";\n"
    result += "; ".join(map(str, N)) + ";\n"
    
    # Write the result to a .txt file
    with open("diffusion_analysis.txt", "w") as file:
        file.write(result)

K = [1e-12, 1e-6, 1e-8, 1e-5, 1e-3, 1e-2, 1e-4, 1e-1]
alpha = [0.1, 0.2, 0.1, 0.2, 0.1, 0.2, 0.1, 0.2]
states = [0, 1, 0, 1, 0, 1, 0, 1]  # States provided as input
num_states = 2
model = "example_model"
analyze_diffusion_coefficients(K, alpha, states, num_states, model)

task -> exp -> fovs

identify the two states (or the oen state) in the experiment
produce the averages and std for each states across all fov files


In [30]:
datasets_phenom().avail_models_name


['single_state',
 'multi_state',
 'immobile_traps',
 'dimerization',
 'confinement']

In [31]:
def pdf(q, bins=100, density=True):
    pdf_q, bins_edge_q = np.histogram(q, bins=bins, density=density)
    bins_center_q = (bins_edge_q[0:-1] + bins_edge_q[1:]) / 2
    return pdf_q, bins_center_q


def K_alpha_mean_and_std(K_array, alpha_array) :
    from scipy.signal import find_peaks
    K_histo, K_bins = pdf(K_array)
    alpha_histo, alpha_bins = pdf(alpha_array)
    peaks_K, _ = find_peaks(K_histo)
    peaks_alpha, _ = find_peaks(alpha_histo)

    peak_indice_K = peaks_K[np.argsort(K_histo[peaks_K])[-2:]]
    peak_indice_alpha = peaks_alpha[np.argsort(alpha_histo[peaks_alpha])[-2:]]


    #maxima_K = K_histo[peak_indice_K]
    peak_means_K = K_bins[peak_indice_K]
    peak_stds_K = []

    peak_means_alpha = alpha_bins[peak_indice_alpha]
    peak_stds_alpha = []

    for peak_idx_K in peak_indice_K:
        # Find the range of indices around the peak
        left_idx_K = np.where(K_bins <= K_bins[peak_idx_K])[0][-1]
        right_idx_K = np.where(K_bins >= K_bins[peak_idx_K])[0][0]
        
        # Calculate standard deviation around the peak
        std_K = np.std(K[(K >= K_bins[left_idx_K]) & (K <= K_bins[right_idx_K])])
        peak_stds_K.append(std_K)

    for peak_idx_alpha in peak_indice_K:
        # Find the range of indices around the peak
        left_idx_alpha = np.where(alpha_bins <= alpha_bins[peak_idx_alpha])[0][-1]
        right_idx_alpha = np.where(alpha_bins >= alpha_bins[peak_idx_alpha])[0][0]
        
        # Calculate standard deviation around the peak
        std_alpha = np.std(alpha[(alpha >= alpha_bins[left_idx_alpha]) & (alpha <= alpha_bins[right_idx_alpha])])
        peak_stds_alpha.append(std_alpha)
    
    return peak_means_K, peak_means_alpha, peak_stds_K, peak_stds_alpha
                                     
    


In [32]:
def find_txt_files_in_subfolders(root_folder):
    # Dictionary to store subfolder paths and their respective txt files
    subfolders_txt_files = {}
    
    for root, dirs, files in os.walk(root_folder):
        for subdir in dirs:
            subdir_path = os.path.join(root, subdir)
            txt_files = []
            for file in os.listdir(subdir_path):
                if file.endswith('.txt'):
                    txt_files.append(file)
            subfolders_txt_files[subdir_path] = txt_files
    
    return subfolders_txt_files

def calculate_stats(file_path):
    data = np.loadtxt(file_path)
    state = data[] #Where ?
    K_mean, alpha_mean, K_std, alpha_std, state = K_alpha_mean_and_std(data[], data[]) #DEFINE THIS ! How can i automatically retrieve k and alpha?
    return K_mean, alpha_mean, K_std, alpha_std, state

def process_txt_files(root_folder):
    txt_files_dict = find_txt_files_in_subfolders(root_folder)

    
    for subfolder, txt_files in txt_files_dict.items():
        K_mean_exp = []
        alpha_mean_exp = []
        K_std_exp = []
        alpha_std_exp = []
        for txt_file in txt_files:
            K_mean, alpha_mean, K_std, alpha_std, state = calculate_stats(txt_file)
            K_mean_exp.append((txt_file, K_mean))
            alpha_mean_exp.append((txt_file, alpha_mean))
            K_std_exp.append((txt_file, K_std))
            alpha_std_exp.append((txt_file, alpha_std))

        K_min = np.mean(K_mean[0,:]) #Unsure about the formatting of this one, check
        K_max = np.mean(K_mean[1,:])
        alpha_min = np.mean(alpha_mean[0,:])
        alpha_max = np.mean(alpha_mean[1,:])
        K_std_min = np.mean(K_std_exp[0,:])
        K_std_max = np.mean(K_std_exp[1,:])
        alpha_std_min = np.mean(alpha_std_exp[0,:])
        alpha_std_max = np.mean(alpha_std_exp[1,:])
    
    
    return K_min, K_max, alpha_min, alpha_max, K_std_min, K_std_max, alpha_std_min, alpha_std_max, state

SyntaxError: invalid syntax (25646118.py, line 18)

In [ ]:
root_folder = './track_2/'
txt_files_dict = find_txt_files_in_subfolders(root_folder)
for subfolder, txt_files in txt_files_dict.items():
    print(f"Subfolder: {subfolder}")
    for txt_file in txt_files:
        print(f"  {txt_file}")

Subfolder: ./track_2/exp_9
  fov_16.txt
  fov_21.txt
  fov_4.txt
  fov_27.txt
  fov_10.txt
  fov_8.txt
  fov_23.txt
  fov_28.txt
  fov_2.txt
  fov_22.txt
  fov_3.txt
  fov_5.txt
  fov_29.txt
  fov_12.txt
  fov_9.txt
  fov_1.txt
  fov_20.txt
  fov_26.txt
  fov_7.txt
  fov_0.txt
  fov_19.txt
  fov_13.txt
  fov_11.txt
  fov_15.txt
  fov_6.txt
  fov_25.txt
  fov_17.txt
  fov_14.txt
  fov_24.txt
  fov_18.txt
Subfolder: ./track_2/exp_11
  fov_16.txt
  fov_21.txt
  fov_4.txt
  fov_27.txt
  fov_10.txt
  fov_8.txt
  fov_23.txt
  fov_28.txt
  fov_2.txt
  fov_22.txt
  fov_3.txt
  fov_5.txt
  fov_29.txt
  fov_12.txt
  fov_9.txt
  fov_1.txt
  fov_20.txt
  fov_26.txt
  fov_7.txt
  fov_0.txt
  fov_19.txt
  fov_13.txt
  fov_11.txt
  fov_15.txt
  fov_6.txt
  fov_25.txt
  fov_17.txt
  fov_14.txt
  fov_24.txt
  fov_18.txt
Subfolder: ./track_2/exp_10
  fov_16.txt
  fov_21.txt
  fov_4.txt
  fov_27.txt
  fov_10.txt
  fov_8.txt
  fov_23.txt
  fov_28.txt
  fov_2.txt
  fov_22.txt
  fov_3.txt
  fov_5.txt
  fov_

In [ ]:
def model_name (state, K_min, K_max, alpha_min, alpha_max):
    if state == 1 :
        return "single_state"
    
    elif state == 2 :
        if K_min == 0.0 :
            return "immobile_traps"
        


In [ ]:
def main_function() :
    root_folder = './track_2/'
    # txt_files_dict = find_txt_files_in_subfolders(root_folder)
    # for subfolder, txt_files in txt_files_dict.items():
    # print(f"Subfolder: {subfolder}")
    # for txt_file in txt_files:
    #     print(f"  {txt_file}")


    K_alpha_stats = process_txt_files(root_folder)
    for exp in range(10):
        # Create the folder of the experiment if it does not exits
        path_exp = "track_2/exp_{exp}/"
        file_name = path_exp + 'ensemble_labels.txt' # ok
            
        with open(file_name, 'a') as f:
            # Save the model (random) and the number of states (2 in this case)
            model_name = np.random.choice(datasets_phenom().avail_models_name, size = 1)[0]
            f.write(f'model: {model_name}; num_state: {2} \n')

            # Create some dummy data for 2 states. This means 2 columns
            # and 5 rows
            data = np.random.rand(5, 2)
                
            data[-1,:] /= data[-1,:].sum()

            # Save the data in the corresponding ensemble file
            np.savetxt(f, data, delimiter = ';')